In [39]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType, IntegerType
from datetime import datetime

In [2]:
spark = SparkSession.builder.master("local[4]").appName("homework").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 08:44:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Q1: Version of spark.


Interestingly, you can do both `spark.version` after defining the spark object. Else run `pyspark.__version__`.

In [6]:
spark.version

'3.4.2'

---

In [24]:
schema_fhv = StructType([
    StructField("dispatching_base_num", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropOff_datetime", TimestampType(), True),
    StructField("PUlocationID", IntegerType(), True),
    StructField("DOlocationID", IntegerType(), True),
    StructField("SR_Flag", StringType(), True),
    StructField("Affiliated_base_number", StringType(), True),
])

df = spark.read.option("header", "true").schema(schema_fhv).csv("./")
df.count()

1898240

In [28]:
df = df.repartition(6)

In [29]:
df.write.parquet('fhv/2019/')

24/03/03 09:11:39 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///home/desnehangsu/notebooks/gather_data.sh
24/03/03 09:11:39 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///home/desnehangsu/notebooks/batch_pyspark.ipynb
24/03/03 09:11:39 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///home/desnehangsu/notebooks/load_to_parq.ipynb
24/03/03 09:11:39 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///home/desnehangsu/notebooks/Untitled.ipynb


In [30]:
data = spark.read.parquet('./fhv/2019/')
data.count()

1898144

In [49]:
q2= data.withColumn("pickup_date", F.to_date('pickup_datetime')).select("pickup_date")
q2 = q2.filter(q2.pickup_date == datetime.strptime('2019-10-15', '%Y-%m-%d').date())

## Q2: No. of trips in Oct 2019

In [50]:
q2_ans

62610

----

In [54]:
data.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



# 04: Highest ride time

In [76]:
q3 = data \
        .withColumn("ride_time", (F.unix_timestamp('dropOff_datetime') - F.unix_timestamp('pickup_datetime'))/3600) \
        .select('pickup_datetime', 'dropOff_datetime', 'ride_time').sort(F.desc('ride_time'))
q3.show(3)

+-------------------+-------------------+-----------------+
|    pickup_datetime|   dropOff_datetime|        ride_time|
+-------------------+-------------------+-----------------+
|2019-10-11 18:00:00|2091-10-11 18:30:00|         631152.5|
|2019-10-28 09:00:00|2091-10-28 09:30:00|         631152.5|
|2019-10-31 23:46:33|2029-11-01 00:13:00|87672.44083333333|
+-------------------+-------------------+-----------------+
only showing top 3 rows



---

In [73]:
schema_lookup = StructType([
    StructField("LocationID", IntegerType(), True),
    StructField("Borough", StringType(), True),
    StructField("Zone", StringType(), True),
    StructField("service_zone", StringType(), True),
])

misc = spark.read.option("header", "true").schema(schema_lookup).csv('taxi_zone_lookup.csv')
misc.schema

StructType([StructField('LocationID', IntegerType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [82]:
q4 = data.join(misc, data.PUlocationID == misc.LocationID, "inner").select(
    "Borough", "Zone", "pickup_datetime")

In [87]:
q4_ans = q4.groupBy("Zone").count().sort(F.asc("count"))

# Q6: Least frequent zone

In [89]:
q4_ans.show(3)

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|         Jamaica Bay|    1|
|Governor's Island...|    2|
| Green-Wood Cemetery|    5|
+--------------------+-----+
only showing top 3 rows

